In [ ]:
from common import *
from matplotlib.offsetbox import AnchoredText

use_hvplot()

In [ ]:
client = start_cluster(16, 24)
client_ip_and_port(client)

## Year

In [ ]:
year = 2021
water_year = f'wy{year}'
# A frist rough filter to reduce number of files read
# Will be refined with below helper `with_snow`
time=slice(f"{year -1}-10-01", f"{year}-07-10")

## Helpers

In [ ]:
def net_lw(sw, lw, em):
    # From pysnobal:
    # https://github.com/USDA-ARS-NWRC/pysnobal/blob/bf8b41c71e3e54ae654ae04005ddf72566c47ee6/pysnobal/c_snobal/libsnobal/_net_rad.c#L36
    lw_out = ((sw.net_solar + 0.98 * lw.thermal - em.net_rad) / 0.98).rename('lw_out')
    return (lw.thermal - lw_out).rename('net_lw')


def snow_dates(em):
    return slice(em.time.min(), em.time.max())

## SNOTEL 

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites.json')

butte_args = dict(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest')
schofield_args = dict(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest')

# iSnobal

Each energy term is aggregated by stattion:
* Time is filtered to only days with non-zero net radiation term
* Forcing data for solar and thermal resampled to daily mean

## Base 

In [ ]:
snobal_em_base = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw/*/em.nc').as_posix(),
    chunks={'time': 744, 'y' :1000, 'x': 1000},
    drop_variables=['cold_content', 'evaporation', 'precip_advected', 'snow_soil', 'snowmelt', 'SWI'],
    parallel=True,
).sel(time=time)

snobal_sw_base = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw/*/smrf_energy_*.nc').as_posix(),
    chunks={'time': 744, 'y' :1000, 'x': 1000},
    drop_variables=['albedo_ir', 'albedo_vis', ],
    parallel=True,
)

snobal_lw_base = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw/*/smrf_20*.nc').as_posix(),
    chunks={'time': 744, 'y' :1000, 'x': 1000},
    drop_variables=['precip_temp', 'percent_snow', 'precip', 'snow_density', 'storm_days', 'wind_speed', 'vapor_pressure', 'air_temp'],
    parallel=True,
)

In [ ]:
butte_em_smrf = snobal_em_base.sel(**butte_args).squeeze(['x', 'y']).compute()
butte_em_smrf = butte_em_smrf.where(butte_em_smrf.net_rad != 0, drop=True).resample(time="1D").mean()
butte_snow_dates = snow_dates(butte_em_smrf)
butte_sw_smrf = snobal_sw_base.sel(time=butte_snow_dates).sel(**butte_args).squeeze(['x', 'y']).compute()
butte_sw_smrf = butte_sw_smrf.where(butte_sw_smrf.net_solar > 0).resample(time="1D").mean()
butte_lw_smrf = snobal_lw_base.sel(time=butte_snow_dates).sel(**butte_args).squeeze(['x', 'y']).compute()
butte_lw_smrf = butte_lw_smrf.resample(time="1D").mean()

schofield_em_smrf = snobal_em_base.sel(**schofield_args).squeeze(['x', 'y']).compute()
schofield_em_smrf = schofield_em_smrf.where(schofield_em_smrf.net_rad != 0, drop=True).resample(time="1D").mean()
schofield_snow_dates = snow_dates(schofield_em_smrf)
schofield_sw_smrf = snobal_sw_base.sel(time=schofield_snow_dates).sel(**schofield_args).squeeze(['x', 'y']).compute()
schofield_sw_smrf = schofield_sw_smrf.where(schofield_sw_smrf.net_solar > 0).resample(time="1D").mean()
schofield_lw_smrf = snobal_lw_base.sel(time=schofield_snow_dates).sel(**schofield_args).squeeze(['x', 'y']).compute()
schofield_lw_smrf = schofield_lw_smrf.resample(time="1D").mean()

In [ ]:
del snobal_em_base
del snobal_sw_base
del snobal_lw_base

## HRRR

In [ ]:
snobal_em_hrrr = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw_hrrr_solar/*/em.nc').as_posix(),
    chunks={'time': 744, 'y' :1000, 'x': 1000},
    drop_variables=['cold_content', 'evaporation', 'precip_advected', 'snow_soil', 'snowmelt', 'SWI'],
    parallel=True,
).sel(time=time)

snobal_sw_hrrr = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw_hrrr_solar/*/net_solar.nc').as_posix(),
    chunks={'time': 744, 'y' :1000, 'x': 1000},
    drop_variables=['DSWRF', 'illumination_angle', 'zenith', 'azimuth', 'albedo_vis', 'albedo_ir'],
    parallel=True,
)

snobal_lw_hrrr = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw_hrrr_solar/*/smrf_20*.nc').as_posix(),
    chunks={'time': 744, 'y' :1000, 'x': 1000},
    drop_variables=['precip_temp', 'percent_snow', 'precip', 'snow_density', 'storm_days', 'wind_speed', 'vapor_pressure', 'air_temp'],
    parallel=True,
)

In [ ]:
butte_em_hrrr = snobal_em_hrrr.sel(**butte_args).squeeze(['x', 'y']).compute()
butte_em_hrrr = butte_em_hrrr.where(butte_em_hrrr.net_rad != 0, drop=True).resample(time="1D").mean()
butte_snow_dates = snow_dates(butte_em_hrrr)
butte_sw_hrrr = snobal_sw_hrrr.sel(time=butte_snow_dates).sel(**butte_args).squeeze(['x', 'y']).compute()
butte_sw_hrrr = butte_sw_hrrr.where(butte_sw_hrrr.net_solar > 0).resample(time="1D").mean()
butte_lw_hrrr = snobal_lw_hrrr.sel(time=butte_snow_dates).sel(**butte_args).squeeze(['x', 'y']).compute()
butte_lw_hrrr = butte_lw_hrrr.resample(time="1D").mean()

schofield_em_hrrr = snobal_em_hrrr.sel(**schofield_args).squeeze(['x', 'y']).compute()
schofield_em_hrrr = schofield_em_hrrr.where(schofield_em_hrrr.net_rad != 0, drop=True).resample(time="1D").mean()
schofield_snow_dates = snow_dates(schofield_em_hrrr)
schofield_sw_hrrr = snobal_sw_hrrr.sel(time=schofield_snow_dates).sel(**schofield_args).squeeze(['x', 'y']).compute()
schofield_sw_hrrr = schofield_sw_hrrr.where(schofield_sw_hrrr.net_solar > 0).resample(time="1D").mean()
schofield_lw_hrrr = snobal_lw_hrrr.sel(time=schofield_snow_dates).sel(**schofield_args).squeeze(['x', 'y']).compute()
schofield_lw_hrrr = schofield_lw_hrrr.resample(time="1D").mean()

In [ ]:
del snobal_em_hrrr
del snobal_sw_hrrr
del snobal_lw_hrrr

## HRRR-MODIS

In [ ]:
snobal_em_hrrr_modis = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw_hrrr_solar_modis_cubic/*/em.nc').as_posix(),
    chunks={'time': 744, 'y' :1000, 'x': 1000},
    drop_variables=['cold_content', 'evaporation', 'precip_advected', 'snow_soil', 'snowmelt', 'SWI'],
    parallel=True,
).sel(time=time)

snobal_sw_hrrr_modis = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw_hrrr_solar_modis_cubic/*/net_solar.nc').as_posix(),
    chunks={'time': 744, 'y' :1000, 'x': 1000 },
    drop_variables=['DSWRF', 'illumination_angle', 'zenith', 'azimuth', 'albedo_vis', 'albedo_ir'],
    parallel=True,
)

snobal_lw_hrrr_modis = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw_hrrr_solar_modis_cubic/*/smrf_20*.nc').as_posix(),
    chunks={'time': 744, 'y' :1000, 'x': 1000 },
    drop_variables=['precip_temp', 'percent_snow', 'precip', 'snow_density', 'storm_days', 'wind_speed', 'vapor_pressure', 'air_temp'],
    parallel=True,
)

In [ ]:
butte_em_hrrr_modis = snobal_em_hrrr_modis.sel(**butte_args).squeeze(['x', 'y']).compute()
butte_em_hrrr_modis = butte_em_hrrr_modis.where(butte_em_hrrr_modis.net_rad != 0, drop=True).resample(time="1D").mean()
butte_snow_dates = snow_dates(butte_em_hrrr_modis)
butte_sw_hrrr_modis = snobal_sw_hrrr_modis.sel(time=butte_snow_dates).sel(**butte_args).squeeze(['x', 'y']).compute()
butte_sw_hrrr_modis = butte_sw_hrrr_modis.where(butte_sw_hrrr_modis.net_solar > 0).resample(time="1D").mean()
butte_lw_hrrr_modis = snobal_lw_hrrr_modis.sel(time=butte_snow_dates).sel(**butte_args).squeeze(['x', 'y']).compute()
butte_lw_hrrr_modis = butte_lw_hrrr_modis.resample(time="1D").mean()

schofield_em_hrrr_modis = snobal_em_hrrr_modis.sel(**schofield_args).squeeze(['x', 'y']).compute()
schofield_em_hrrr_modis = schofield_em_hrrr_modis.where(schofield_em_hrrr_modis.net_rad != 0, drop=True).resample(time="1D").mean()
schofield_snow_dates = snow_dates(schofield_em_hrrr_modis)
schofield_sw_hrrr_modis = snobal_sw_hrrr_modis.sel(time=schofield_snow_dates).sel(**schofield_args).squeeze(['x', 'y']).compute()
schofield_sw_hrrr_modis = schofield_sw_hrrr_modis.where(schofield_sw_hrrr_modis.net_solar > 0).resample(time="1D").mean()
schofield_lw_hrrr_modis = snobal_lw_hrrr_modis.sel(time=schofield_snow_dates).sel(**schofield_args).squeeze(['x', 'y']).compute()
schofield_lw_hrrr_modis = schofield_lw_hrrr_modis.resample(time="1D").mean()

In [ ]:
del snobal_em_hrrr_modis
del snobal_sw_hrrr_modis
del snobal_lw_hrrr_modis

# EM In- and Out-Energy

### Helpers 

In [ ]:
def with_eb(data, em):
    return data.where(~em.sum_EB.isnull())

def no_snow(em):
    return em.sum_EB.where(em.sum_EB.isnull(), 0, True).fillna(0)

In [ ]:
def em_plots(em, lw, sw, label):
    lw_net = net_lw(sw, lw, em)
    
    return with_eb(sw.net_solar, em).hvplot(label='Net Solar').opts(color=hv.Palette('Colorblind'), **HV_PLOT_OPTS, title=label) * \
           with_eb(em.net_rad, em).hvplot(label='Net Radiation') * \
           with_eb(lw_net, em).hvplot(label='Net LW') * \
           with_eb(em.sensible_heat, em).hvplot(label='Sensible Heat') * \
           with_eb(em.latent_heat, em).hvplot(label='Latent Heat') * \
           with_eb(em.sum_EB, em).hvplot(label='Sum EB')

def em_plot_mpl_stacked(top, bottom, title):
    figure_opts = dict(figsize=(7,6), dpi=300,)
    fig, axes = plt.subplots(2, 1, sharex=True, **figure_opts)
    plt.subplots_adjust(hspace=0.075)
    
    em_plots_mpl(top[0], top[1], top[2], 'Butte', axes[0])
    em_plots_mpl(bottom[0], bottom[1], bottom[2], 'Schofield Pass', axes[1])
    
    axes[1].legend(
        frameon=False,
        loc='lower center',
        bbox_to_anchor=(0.5, -0.3),
        ncol=4,
        borderaxespad=0.15, 
        fontsize=8
    )
    at = AnchoredText(
        title, 
        prop=dict(size=9, weight='bold'), 
        frameon=False, 
        loc='upper center', 
        pad=0.3, 
        borderpad=0.25,
    )
    axes[0].add_artist(at)

def em_plots_mpl(em, lw, sw, label, ax, single=False):
    if single:
        figure = plt.figure(layout='constrained', figsize=(10, 4), dpi=300)
        ax = figure.gca()
    
    lw_net = net_lw(sw, lw, em)
    
    plot_opts = dict(
        ax=ax,
        lw=0.95,
        alpha=0.85
    )
    
    with_eb(sw.net_solar, em).plot(label='Net Solar', **plot_opts, color='royalblue')
    with_eb(em.net_rad, em).plot(label='Net Radiation', **plot_opts, color='firebrick')
    with_eb(lw_net, em).plot(label='Net Longwave', **plot_opts, color='tan')
    with_eb(em.sum_EB, em).plot(label='Sum of Energy', **plot_opts, color='blueviolet')
    with_eb(em.sensible_heat, em).plot(label='Sensible Heat', **plot_opts, color='orange')
    with_eb(em.latent_heat, em).plot(label='Latent Heat', **plot_opts, color='seagreen')
    
    ns = no_snow(em)
    ax.scatter(ns.time.values, ns.values, label='No Snow', color='grey', marker='.', alpha=0.75, s=35, edgecolors='none')
    
    ax.axvspan(np.datetime64(f'{year - 1}-10-01'), em.time.min().values, color='grey', alpha=0.2)
    ax.axvspan(em.time.max().values, np.datetime64(f'{year}-07-10'), color='grey', alpha=0.2)
    
    ax.set_title('')
    ax.set_ylabel(r'$W/m^2$')
    ax.set_xlabel('')
    ax.set_xlim(np.datetime64(f'{year - 1}-10-01'), np.datetime64(f'{year}-07-10'))
    ax.set_xticks(
        ax.get_xticks(),
        [f'{year-1}-10', '11', '12', f'{year}-01', '02', '03', '04', '05', '06', '07'],
        rotation=0, ha='center', va='top', fontsize='small'
    )
    ax.set_ylim(-100, 200)
    ax.set_yticks(
        ax.get_yticks(),
        ['', -50, 0, 50, 100, 150, ''],
        fontsize='small',
    )
                 
    if label:
        at = AnchoredText(
            label, 
            prop=dict(size='small', style='italic'), 
            frameon=True, 
            loc='upper left', 
            pad=0.3, 
            borderpad=0.25,
        )
        at.patch.set_boxstyle("round", pad=0., rounding_size=0.2)
        at.patch.set(edgecolor='none', alpha=0.7)
        ax.add_artist(at)


## Base

In [ ]:
em_plot_mpl_stacked(
    [butte_em_smrf, butte_lw_smrf, butte_sw_smrf],
    [schofield_em_smrf, schofield_lw_smrf, schofield_sw_smrf],
    'SMRF'
)

### Butte

In [ ]:
em_plots(butte_em_smrf, butte_lw_smrf, butte_sw_smrf, 'Butte')

In [ ]:
# em_plots_mpl(butte_em_smrf, butte_lw_smrf, butte_sw_smrf, 'Butte', None, True)

### Schofield Pass 

In [ ]:
em_plots(schofield_em_smrf, schofield_lw_smrf, schofield_sw_smrf, 'Schofield Pass')

In [ ]:
# em_plots_mpl(schofield_em_smrf, schofield_lw_smrf, schofield_sw_smrf, 'Schofield Pass', None, True)

## HRRR

In [ ]:
em_plot_mpl_stacked(
    [butte_em_hrrr, butte_lw_hrrr, butte_sw_hrrr],
    [schofield_em_hrrr, schofield_lw_hrrr, schofield_sw_hrrr],
    'HRRR-SC'
)

### Butte

In [ ]:
em_plots(butte_em_hrrr, butte_lw_hrrr, butte_sw_hrrr, 'Butte')

In [ ]:
# em_plots_mpl(butte_em_hrrr, butte_lw_hrrr, butte_sw_hrrr, 'Butte', None, True)

### Schofield 

In [ ]:
em_plots(schofield_em_hrrr, schofield_lw_hrrr, schofield_sw_hrrr, 'Schofield Pass')

In [ ]:
# em_plots_mpl(schofield_em_hrrr, schofield_lw_hrrr, schofield_sw_hrrr, 'Schofield Pass', None, True)

### HRRR-MODIS

In [ ]:
em_plot_mpl_stacked(
    [butte_em_hrrr_modis, butte_lw_hrrr_modis, butte_sw_hrrr_modis],
    [schofield_em_hrrr_modis, schofield_lw_hrrr_modis, schofield_sw_hrrr_modis],
    'HRRR-MODIS'
)

### Butte

In [ ]:
em_plots(butte_em_hrrr_modis, butte_lw_hrrr_modis, butte_sw_hrrr_modis, 'Butte')

In [ ]:
# em_plots_mpl(butte_em_hrrr_modis, butte_lw_hrrr_modis, butte_sw_hrrr_modis, 'Butte', None, True)

### Schofield 

In [ ]:
em_plots(schofield_em_hrrr_modis, schofield_lw_hrrr_modis, schofield_sw_hrrr_modis, 'Schofield Pass')

In [ ]:
# em_plots_mpl(schofield_em_hrrr_modis, schofield_lw_hrrr_modis, schofield_sw_hrrr_modis, 'Schofield Pass', None, True)

# Difference in EM 

In [ ]:
def em_diff_plots(em_base, lw_base, sw_base, em, lw, sw, label):
    lw_base_net = net_lw(sw_base, lw_base, em_base)
    lw_net = net_lw(sw, lw, em)
    
    sw_net_diff = (sw.net_solar - sw_base.net_solar).rename('Net Solar diff')
    lw_net_diff = (lw_net - lw_base_net).rename('Net LW diff')
    net_rad_diff = (em.net_rad - em_base.net_rad).rename('Net Rad diff')

    return sw_net_diff.hvplot(label='Net Solar Diff', color='steelblue').opts(**HV_PLOT_OPTS, title=label) * \
            hv.Area(net_rad_diff, label='Net Rad Diff').opts(color='coral', alpha=0.9) * \
            lw_net_diff.hvplot(label='Net LW Diff', color='olive', alpha=0.9)

## HRRR
### Butte

In [ ]:
em_diff_plots(
    butte_em_smrf, butte_lw_smrf, butte_sw_smrf, 
    butte_em_hrrr, butte_lw_hrrr, butte_sw_hrrr, 
    'Butte'
)

### Schofield Pass

In [ ]:
em_diff_plots(
    schofield_em_smrf, schofield_lw_smrf, schofield_sw_smrf, 
    schofield_em_hrrr, schofield_lw_hrrr, schofield_sw_hrrr, 
    'Schofield Pass'
)

## HRRR MODIS
### Butte

In [ ]:
em_diff_plots(
    butte_em_smrf, butte_lw_smrf, butte_sw_smrf, 
    butte_em_hrrr_modis, butte_lw_hrrr_modis, butte_sw_hrrr_modis, 
    'Butte'
)

### Schofield Pass 

In [ ]:
em_diff_plots(
    schofield_em_smrf, schofield_lw_smrf, schofield_sw_smrf, 
    schofield_em_hrrr_modis, schofield_lw_hrrr_modis, schofield_sw_hrrr_modis, 
    'Schofield Pass'
)

# EM Differences

## NOTE

Difference always subtract SMRF as the baseline run
* Net Radiation (**NR**)
* Sensible Heat (**SH**)
* Latent Heat (**LH**)

In [ ]:
from matplotlib.lines import Line2D

In [ ]:
def box_plot_data(em_base, lw_base, sw_base, em, lw, sw):
    lw_base_net = net_lw(sw_base, lw_base, em_base)
    lw_net = net_lw(sw, lw, em)
    
    lw_net_diff = (lw_net - lw_base_net).dropna(dim='time')
    sw_net_diff = (sw.net_solar - sw_base.net_solar).dropna(dim='time')
    sum_EB_diff = (em.sum_EB - em_base.sum_EB).dropna(dim='time')
        
    print("** Sum EB")
    print(f"  mean: {sum_EB_diff.mean():.2f}")
    print(f"  std: {sum_EB_diff.std():.2f}")
    print(f"  median: {sum_EB_diff.median():.2f}")
    print("** Net SW")
    print(f"  mean: {sw_net_diff.where(sw_net_diff > 0).mean():.2f}")
    print(f"  std: {sw_net_diff.where(sw_net_diff > 0).std():.2f}")
    print(f"  median: {sw_net_diff.where(sw_net_diff > 0).median():.2f}")
    print("** Net LW")
    print(f"  mean: {lw_net_diff.mean():.2f}")
    print(f"  std: {lw_net_diff.std():.2f}")
    print(f"  median: {lw_net_diff.median():.2f}")
    
    return [
        (em.sensible_heat - em_base.sensible_heat).dropna(dim='time').values,
        (em.latent_heat - em_base.latent_heat).dropna(dim='time').values,
        lw_net_diff.values,
        sw_net_diff.values,
        sum_EB_diff.values,
    ]

def style_vp(vp):
    idx = 0
    for pc in vp['bodies']:
        if idx == 4:
            pc.set_facecolor('orange')
        else:
            pc.set_facecolor('lightsteelblue')
        pc.set_edgecolor('Black')
        pc.set_lw(0.75)
        idx += 1

    for line in ['cquantiles']: #, 'cmeans', 'cmedians']:
        vp[line].set_color('black')
        vp[line].set_lw(1.5)

    # vp['cmedians'].set_ls(':')
    # vp['cmedians'].set_lw(2)
    # vp['cmeans'].set_lw(2)


def box_figure(left_data, right_data):
    figure = plt.figure(layout='constrained', figsize=(12, 5), dpi=300)
    (left, right) = figure.subfigures(nrows=1, ncols=2, wspace=-1)
    # plt.subplots_adjust(wspace=0.05)
    
    left_axes = left.subplots(1, 2, sharey=True,)
    right_axes = right.subplots(1, 2, sharey=True,)

    left.suptitle('HRRR-SC', style='italic', weight='bold')
    print('HRRR SC')
    boxplot(left_data[0], left_data[1], left_axes, True)
    
    right.suptitle('HRRR-MODIS', style='italic', weight='bold')
    print('\n\nHRRR MODIS')
    boxplot(right_data[0], right_data[1], right_axes)
    
    figure.suptitle(f'Water Year {year}')
            
    figure.legend(
        handles=[
            # Line2D([0], [0], color='black', linewidth=2, linestyle=':', label='Means'),
            # Line2D([0], [0], color='black', linewidth=3, label='Median'),
            Line2D([0], [0], color='black', linewidth=1.5, label='Quantiles (95%, 5%)')
        ],
        loc='lower right',
        fontsize=8,
        frameon=False,
        bbox_to_anchor=(0.45, 0.9),
        ncol=1
    )
    

def boxplot(butte, schofield, axes, y_label=False):
    (ax1, ax2) = axes
    
    labels = ['Sensible\nHeat', 'Latent\nHeat', 'Net\nLongwave', 'Net\nShortwave', 'Sum of\nEnergy']
    labels = ['SH', 'LH', 'NL', 'NS', 'Q']

    violin_style = dict(
        widths=0.5,
        showmeans=False,
        showmedians=False,
        showextrema=False,
        positions=np.arange(1, 6, step=1),
        quantiles=[
           [0.05, 0.95],
           [0.05, 0.95],
           [0.05, 0.95],
           [0.05, 0.95],
           [0.05, 0.95]
        ]        
    )
    
    print("* Butte")
    vp = ax1.violinplot(
        box_plot_data(
            butte['em_base'], butte['lw_base'], butte['sw_base'], 
            butte['em'], butte['lw'], butte['sw']
        ),
        **violin_style,
    )
    style_vp(vp)
    ax1.set_xticks(violin_style['positions'], labels)   
    
    if y_label:
        ax1.set_ylabel(r'$\Delta$ $W/m^2$')
        ax1.legend(
        handles=[
            # Line2D([0], [0], color='black', linewidth=2, linestyle=':', label='Means'),
            # Line2D([0], [0], color='black', linewidth=3, label='Median'),
            Line2D([0], [0], color='black', linewidth=1.5, label='Quantiles (95%, 5%)')
        ],
        loc='lower right',
        fontsize=8,
        frameon=False,
        bbox_to_anchor=(0.65, 0.9),
        ncol=1
    )
    
    print("\n* Schofield Pass")
    vp = ax2.violinplot(
        box_plot_data(
            schofield['em_base'], schofield['lw_base'], schofield['sw_base'], 
            schofield['em'], schofield['lw'], schofield['sw']
        ),
        **violin_style
    )
    style_vp(vp)
    ax2.set_xticks(violin_style['positions'], labels)
    
    ax1.yaxis.grid(True)
    ax1.set_title('Butte', size='medium')
    ax2.yaxis.grid(True)
    ax2.set_title('Schofield Pass', size='medium')

    ax1.set_ylim(-75, 125)

Includes ground heat flux and precip advection

In [ ]:
box_figure(
    [
        {
            'em_base': butte_em_smrf, 'lw_base': butte_lw_smrf, 'sw_base': butte_sw_smrf,
            'em': butte_em_hrrr, 'lw': butte_lw_hrrr, 'sw': butte_sw_hrrr,     
        },
        {
            'em_base': schofield_em_smrf, 'lw_base': schofield_lw_smrf, 'sw_base': schofield_sw_smrf,
            'em': schofield_em_hrrr, 'lw': schofield_lw_hrrr, 'sw': schofield_sw_hrrr,         
        }
    ],
    [
        {
            'em_base': butte_em_smrf, 'lw_base': butte_lw_smrf, 'sw_base': butte_sw_smrf,
            'em': butte_em_hrrr_modis, 'lw': butte_lw_hrrr_modis, 'sw': butte_sw_hrrr_modis,     
        },
        {
            'em_base': schofield_em_smrf, 'lw_base': schofield_lw_smrf, 'sw_base': schofield_sw_smrf,
            'em': schofield_em_hrrr_modis, 'lw': schofield_lw_hrrr_modis, 'sw': schofield_sw_hrrr_modis,         
        }
    ]
)

In [ ]:
def stack_plot(data, base, label):
    overlay = hv.Overlay([
        hv.Area((data.net_rad - base.net_rad), label='Net Rad'), 
        hv.Area(-(data.sensible_heat - base.sensible_heat), label='Sensible Heat'),
        hv.Area(-(data.latent_heat - base.latent_heat), label='Latent Heat'),
    ])
    return hv.Area.stack(overlay).opts(**HV_PLOT_OPTS, title=label)

## Butte 

In [ ]:
stack_plot(butte_em_hrrr, butte_em_smrf, 'HRRR')

In [ ]:
stack_plot(butte_em_hrrr_modis, butte_em_smrf, 'HRRR-MOIDS')

## Schofield Pass 

In [ ]:
stack_plot(schofield_em_hrrr, schofield_em_smrf, 'HRRR')

In [ ]:
stack_plot(schofield_em_hrrr_modis, schofield_em_smrf, 'HRRR-MODIS')

In [ ]:
client.shutdown()